# Leja points in PINNs

## Introduction

This notebook will perform the general setup to solve the homogenous heat equation \begin{align*} &&&&u_t &= \Delta u &&&&&& \\
&&&&u(x,0) &= -\prod\limits_{i=1}^d \sin (\pi x_i) && \forall x\in D &&&& \\
&&&&u(x,t) &= 0 && \forall x\in \partial D, \: t\in(0,1). &&&& \end{align*} on $D=[-1,1]^d$ by a neuronal network $u_{\theta}$. For this, the loss function will consist of the residuals of $u_{\theta}$ with respect to the equation as well as initial and boundary condition. Their values are obtained by virtue of quadrature formulas over the respective domains.

Along the way several hyperparameters (network size, optimizer, weighting of the loss function) can be changed to observe their influence on the error of $u_{\theta}$ with respect to the exact solution $$u(x,t) = - e^{- d \pi^2 t} \prod\limits_{i=1}^d \sin (\pi x_i).$$

Of particular interest is the convergence of the PINN with respect to the number of training points when using different quadrature formulas to obtain the loss function. Those quadrature formulas used are Monte Carlo as well as a product- and a sparse grid based on leja-points.

### How to use this notebook

First, execute the code cells from General setup in sequential order. Depending on the testing to be done, not all of them may be necessary. In particular, testing hyperparameters only uses the product grid.

Next, decide which test will be done. If it is one of the hyperparameter ones, first execute the general cell that sets up the quadrature formula for those and then the cells in the section you want to do. If it is for the quadrature formulas, only execute the code in that section.

## General setup

First, let's import the relevant packages and set some general parameters.

In [ ]:
import tensorflow as tf #used for handling neural networks
import numpy as np      #generally useful

xdims =  1              #Number of (spatial) dimensions, called d in the theory

DTYPE='float64'         #Data type to be used in computations
tf.keras.backend.set_floatx(DTYPE)

tf.random.set_seed(0)   #Make randomness consistent across runs

pi = tf.constant(np.pi, dtype=DTYPE)  #Will be used shortly

Next we will set up the differential equation including initial and boundary conditions as well as its domain $[0,1] \times [-1,1]^d$. The equation takes the form \begin{align*} res(u) &\equiv 0 &&\\ u(x,0) &= u_0(x) && \forall \: x\\  u(x,t) &= u_b(x,t) \quad &&\mbox{on the boundary} .\end{align*} In particular, we arrive at the homogenous heat equation by setting $$res(u) = u_t - \Delta u.$$ We also choose homogenous boundary conditions ($u_b \equiv 0$) and initial data $$u_0(x) = -\prod\limits_{i=1}^d \sin (\pi x_i).$$ Also note that the vector $x$ passed to these functions also contains $t$ as its first element (and thus has length d+1).

In [ ]:
def fun_u_0(x):                            #initial data
    n = x.shape[0]
    val = - tf.ones((n,1), dtype=DTYPE)
    for i in range(xdims):
        x_cur = x[:,(i+1):(i+2)]
        val = tf.math.multiply(val,tf.sin(pi*x_cur))  #here: a sine function
    return val

def fun_u_b(x):                            #boundary data
    n = x.shape[0]
    return tf.zeros((n,1), dtype=DTYPE)    #here: all zeros

def fun_res(u_t,u_xx):                     #residual of the differential equation depending on some derivatives
    res = u_t                              #here: the heat equation
    for i in range(u_xx.shape[1]):
        res -= u_xx[:,i:(i+1)]
    return res


lbt = tf.constant(0, dtype=DTYPE)     #lower bound for t (time)
lbx = tf.constant(-1, dtype=DTYPE)    #lower bound for all x (spatial dimensions)
ubt = tf.constant(1, dtype=DTYPE)     #upper bound for t (time)
ubx = tf.constant(1, dtype=DTYPE)     #upper bound for all x (spatial dimensions)


lb_list = []              #we also want to put all constraints into one vector
ub_list = []
lb_list.append(0)         #adding time constraints
ub_list.append(1)

for _ in range(xdims):
    lb_list.append(-1)    #adding spatial constraints
    ub_list.append(1)

lb = tf.constant(lb_list, dtype=DTYPE, shape=[xdims+1])  #all lower bounds
ub = tf.constant(ub_list, dtype=DTYPE, shape=[xdims+1])  #all upper bounds

Also, the network gets created here. The default is a FNN with 4 hidden layers and 10 neurons per layer.

In [ ]:
def init_model(num_hidden_layers=4, num_neurons_per_layer=10):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.InputLayer(xdims+1))                #input layer

    scaling_layer = tf.keras.layers.Lambda(
                lambda x: 2.0*(x - lb)/(ub - lb) - 1.0)
    model.add(scaling_layer)

    for _ in range(num_hidden_layers):
        model.add(tf.keras.layers.Dense(num_neurons_per_layer,    #hidden layers
            activation=tf.keras.activations.get('tanh'),
            kernel_initializer='glorot_normal'))
        
    model.add(tf.keras.layers.Dense(1))                           #output layer
    
    return model

### Computing loss and its gradient

The next functions deal with computing the loss of a model. The derivatives necessary for computing the residual in the training points $X_r$ are taken using tensorflows inbuilt automatic differentiation. Also, the residuals of the initial and boundary condition in their training points $X_{data}$ are computed. Then, the weights from the chosen quadrature formulas (defined later) using $X_r$ and $X_{data}$ as nodes are applied to all three residuals. Finally, the loss is the (possibly weighted) sum of these.

In [ ]:
def get_res_int(model, X_r):   #X_r are the points where the residual gets evaluated
    xvecs = []
    u_x_vecs = []
    u_xx_vecs = []
    
    with tf.GradientTape(persistent=True) as tape:  #tape will record actions for automatic differentiation
        t = X_r[:, 0:1]
        tape.watch(t)
        for i in range(xdims):
            xvecs.append(X_r[:,(i+1)])     #separate the variables
            tape.watch(xvecs[i])           #and have them watched
        
        u = model(tf.concat([t, tf.stack(xvecs, axis=1)], 1))  #the function we want the derivative of
        for i in range(xdims):
            u_x_vecs.append(tape.gradient(u, xvecs[i]))        #derivatives by all spatial variables

            
    u_t = tape.gradient(u, t)           #derivative by time
    for i in range(xdims):                                      #since first derivatives were computed inside the
        u_xx_vecs.append(tape.gradient(u_x_vecs[i], xvecs[i]))  #context of the tape, we can take second derivatives

    u_xx = tf.stack(u_xx_vecs, axis=1)

    del tape

    return fun_res(u_t, u_xx)

    
def get_res_vectors(model, X_r, X_data, u_data):  #X_data are the points where initial and boundary conditions
                                                  #are to be evaluated. u_data are the expected values
    res_int = get_res_int(model, X_r)

    u_pred = model(X_data[0])         #for initial condition
    res_tb = u_data[0]-u_pred
    
    u_pred = model(X_data[1])         #for boundary condition
    res_sb = u_data[1]-u_pred

    return res_int, res_sb, res_tb


def get_training_errs(model, X_r, w_r, X_data, u_data, w_data):  #this applies the respective quadrature formulas
                                                                 #to the residual vectors previously computed
    res_int, res_sb, res_tb = get_res_vectors(model, X_r, X_data, u_data)

    err_int = tf.reduce_mean(tf.math.multiply(w_r,tf.square(res_int)))
    err_sb = tf.reduce_mean(tf.math.multiply(w_data[1],tf.square(res_sb)))
    err_tb = tf.reduce_mean(tf.math.multiply(w_data[0],tf.square(res_tb)))

    return err_int, err_sb, err_tb


def compute_loss(model, X_r, w_r, X_data, u_data, w_data, lam_int, lam_sb, lam_tb):
    err_int, err_sb, err_tb = get_training_errs(model, X_r, w_r, X_data, u_data, w_data)
    
    #add up the single training errors to get the final loss
 
    return lam_int * err_int + lam_sb * err_sb + lam_tb * err_tb

For the optimization, we will need to differentiate this loss function by the weights and biases (trainable variables) of the model. Once again, automatic differentiation is used.

In [ ]:
def get_grad(model, X_r, w_r, X_data, u_data, w_data, lam_int, lam_sb, lam_tb):
    
    with tf.GradientTape(persistent=True) as tape:
        tape.watch(model.trainable_variables)           #watch weights and biases
        loss = compute_loss(model, X_r, w_r, X_data, u_data, w_data, lam_int, lam_sb, lam_tb)

    g = tape.gradient(loss, model.trainable_variables)  #get the desired gradient
    del tape

    return loss, g

### Error estimation

The error of the PINN will later need to be measured. For this, we set up a quadrature formula distinct from the ones used in the PINN and compute the value of the exact solution $$u(x,t) = - e^{- d \pi^2 t} \prod\limits_{i=1}^d \sin (\pi x_i)$$ in its nodes.

In [ ]:
N_test_per_dim = 101                #Size of the grid, this is propably overkill
N_test = N_test_per_dim**(xdims+1)
pts_t, wts_t = np.polynomial.legendre.leggauss(N_test_per_dim)  #get points and weights for Gauss-Legendre-Quadrature
pts_t = 0.5 * (pts_t + 1)   #scale to the time interval
wts_t = 0.5 * wts_t

pts_x, wts_x = np.polynomial.legendre.leggauss(N_test_per_dim)  #points and weights for the first spatial dimension


test_pts_t, test_pts_x = np.meshgrid(pts_t,pts_x)  #constructing a product grid from these points
test_pts_t = test_pts_t.flatten()
test_pts_x = test_pts_x.flatten()

test_wts = np.outer(wts_t,0.5*wts_x)
test_wts = test_wts.flatten()


test_pts_list = [test_pts_t, test_pts_x]

for _ in range(xdims-1):                 #the same procedure for all subsequent spatial dimensions
    new_test_pts_list = []
    pts_to_add, wts_to_add = np.polynomial.legendre.leggauss(N_test_per_dim)
    for pts in test_pts_list:        
        old_pts,new_pts = np.meshgrid(pts,pts_to_add)
        old_pts = old_pts.flatten()
        new_test_pts_list.append(old_pts)

    new_pts = new_pts.flatten()
    new_test_pts_list.append(new_pts)
    test_pts_list = new_test_pts_list

    test_wts = np.outer(test_wts,0.5*wts_to_add)
    test_wts = test_wts.flatten()

test_pts_np = np.stack(test_pts_list,axis=1)
test_pts = tf.constant(test_pts_np, dtype=DTYPE)

test_vals = - np.exp((-np.pi)*np.pi*xdims*test_pts_np[:,0])     #the exact solution is known explicitly here
for i in range(xdims):
    test_vals = test_vals * np.sin(np.pi*test_pts_np[:,(i+1)])

test_vals = tf.reshape(test_vals,[N_test,1])

Using this quadrature formula, we can now estimate the 2-norm of the error of any model (PINN).

In [ ]:
def get_error(model):
    model_vals = model(test_pts)
    err_vec = model_vals-test_vals   
    
    err_two = np.sqrt(np.sum(err_vec*err_vec*tf.constant(test_wts, shape=(N_test,1), dtype=DTYPE)))
        
    return err_two

### Optimization

Here, we set the parameters for the two optimizers Adam and L-BFGS-B as well as when to switch from the former to the latter. Most important here is Adams learning rate $lr_{adam}$ and the tolerance $ftol$ of BFGS.

In [ ]:
lr_adam = 1e-2        #learning rate of Adam
e_target_adam = 1e-2  #loss to be reached with Adam before switch to BFGS
N_adam = 1000        #after how many iterations it will switch regardless of loss

boundaries = [0]            #to be fed as parameters into tensorflows implementation of Adam
values = [lr_adam,lr_adam]

lr = tf.keras.optimizers.schedules.PiecewiseConstantDecay(boundaries,values)  #setting constant learning rate
optim = tf.keras.optimizers.Adam(learning_rate=lr)

opt_opt = {'maxiter': 50000, 'maxfun': 50000, 'maxcor': 50, 'maxls': 50, 'ftol': 1e-8} #options that will be used
                                                                                       #with L-BFGS-B

It is straightforward to use optimizers implemented in tensorflow (like Adam). Also note that all weights of the loss function are set to one by default.

In [ ]:
def train_step(model, X_r, w_r, X_data, u_data, w_data, lam_int=1, lam_sb=1, lam_tb=1):
    loss, grad_theta = get_grad(model, X_r, w_r, X_data, u_data, w_data, lam_int, lam_sb, lam_tb)
    
    optim.apply_gradients(zip(grad_theta, model.trainable_variables))
    
    return loss

We will also make use of scipy's implementation of L-BFGS-B. This needs a way to interface with our tensors.

In [ ]:
from scipy.optimize import minimize

def train_with_scipy_optimizer(model, X_r, w_r, X_data, u_data, w_data, hist, err_hist="a", lam_int=1, lam_sb=1, lam_tb=1):

    it = 0
    current_loss = 0    

    weight_list = []
    shape_list = []
            
    for v in model.variables:
        shape_list.append(v.shape)
        weight_list.extend(v.numpy().flatten())
                
    x0 = tf.convert_to_tensor(weight_list)

    def set_weight_tensor(weight_list):
        idx = 0
        for v in model.variables:
            vs = v.shape
                
            if len(vs) == 2:  
                sw = vs[0]*vs[1]
                new_val = tf.reshape(weight_list[idx:idx+sw],(vs[0],vs[1]))
                idx += sw
                
            elif len(vs) == 1:
                new_val = weight_list[idx:idx+vs[0]]
                idx += vs[0]
                    
            elif len(vs) == 0:
                new_val = weight_list[idx]
                idx += 1
                    
            v.assign(tf.cast(new_val, DTYPE))

    def get_loss_and_grad(w):   #loss and its gradient to be used by the optimizer
  
        set_weight_tensor(w)
        
        loss, grad = get_grad(model, X_r, w_r, X_data, u_data, w_data, lam_int, lam_sb, lam_tb)
                       
        loss = loss.numpy().astype(np.float64)
        hist.append(loss)                        #loss history is saved for later reference
        if err_hist != "a":
            err_hist.append(get_error(model))        #so is the (estimated) error

        grad_flat = []
        for g in grad:
            grad_flat.extend(g.numpy().flatten())
            
        grad_flat = np.array(grad_flat,dtype=np.float64)
            
        return loss, grad_flat

    def callback(xk):           #occasionally output current progress
        if len(hist)%200 == 0:
            print('It {:05d}: loss = {:6.4e}'.format(len(hist),hist[-1]))
        

    minimize(fun=get_loss_and_grad, x0=x0, jac=True, method='L-BFGS-B', callback=callback, options=opt_opt)

### Quadrature formulas

Finally, let's prepare some things for our quadrature formulas. For Monte-Carlo we want to make sure that each face of the boundary has the appropriate number of points. This $N_{bd}$ will later be determined by the number of points on the boundary in the grids.

In [ ]:
def random_boundary_pts(boundary_dim, N_bd):

    pts = tf.random.uniform((N_bd,1), lbt, ubt, dtype=DTYPE) 

    if boundary_dim>0:
        pts = tf.concat([pts, tf.random.uniform((N_bd,boundary_dim), lbx, ubx, dtype=DTYPE)],1) 
  
    #The following is the variable for which the points are at the boundary. Therefore we want its value to be
    #lbx (=-1) or ubx (=1). All other variables are uniformly distributed.
    pts = tf.concat([pts, lbx + (ubx - lbx) * tf.keras.backend.random_bernoulli((N_bd,1), 0.5, dtype=DTYPE)], 1)

    if boundary_dim<(xdims-1):
        pts = tf.concat([pts, tf.random.uniform((N_bd,xdims-boundary_dim-1), lbx, ubx, dtype=DTYPE)],1)

    return pts

As a building block for the product grids, we define a function that outputs one column of a product grid in variable dimension. This will be used in $d$ dimensions to build quadrature rules over the boundary and in $d+1$ dimensions for quadrature over the entire domain.

In [ ]:
import chaospy as cp #used here to get leja-points + quadrature weights

def leja_tensor_pts(col, dims, level):
    needed_leja,needed_wts = cp.quadrature.leja(level-1,cp.Uniform(-1,1))  #the correct amount of points (and
                                                                           #corresponding quadrature weights)
    pts_prep = []
    wts = []

    for _ in range(level**col):
        pts_prep.append(np.repeat(needed_leja,level**(dims-col-1)))
        wts.extend(np.repeat(needed_wts,level**(dims-col-1)))

    return tf.constant(pts_prep, dtype=DTYPE, shape=[level**dims,1]),wts

In particular, it can be directly used to get the correct points on the boundary for this case. For this, seperate grids on the faces of the boundary are constructed and then combined.

In [ ]:
def leja_tensor_boundary_pts(boundary_dim, level):
    base_pts, wts = leja_tensor_pts(0,xdims,level)
    wts = np.array(wts)
    pts = (lbt+(ubt - lbt)/2) + (ubt - lbt)/2 * base_pts

    for i in range(boundary_dim):
        new_pts, new_wts = leja_tensor_pts(i+1,xdims,level)
        new_wts = np.array(new_wts)
        pts = tf.concat([pts, (lbx+(ubx - lbx)/2) + (ubx - lbx)/2 * new_pts], 1)
        wts = wts * new_wts
        
    #Points will lie on the upper and lower boundary for this variable. As far as all other variables are concerned,
    #a normal product grid is constructed.
    pts_ub = tf.concat([pts, ubx * tf.ones((level**xdims,1), dtype=DTYPE)], 1)
    pts_lb = tf.concat([pts, lbx * tf.ones((level**xdims,1), dtype=DTYPE)], 1)

    for i in range(boundary_dim+1, xdims):
        new_pts, new_wts = leja_tensor_pts(i,xdims,level)
        new_wts = np.array(new_wts)
        pts_lb = tf.concat([pts_lb, (lbx+(ubx - lbx)/2) + (ubx - lbx)/2 * new_pts], 1)
        pts_ub = tf.concat([pts_ub, (lbx+(ubx - lbx)/2) + (ubx - lbx)/2 * new_pts], 1)
        wts = wts * new_wts

    wts = np.append(wts,wts,0) * 0.5 #Two grids have been constrructed at the same time (for opposing faces of the boundary).
                                     #Their weights are identical but need to be scaled down accordingly

    return tf.concat([pts_lb, pts_ub],0), wts

Everything for the sparse grids will be done in Matlab and simply imported here.

In [ ]:
import matlab.engine

def leja_sparse_pts(level):    
    
    eng = matlab.engine.start_matlab()                        #start a matlab instance
    X_r,w_r = eng.leja_quad(level,xdims,nargout=2)            #grid over the domain
    X_b,w_b = eng.leja_quad_boundary(level+2,xdims,nargout=2) #grids on the boundary
    X_0,w_0 = eng.leja_quad_initial(level+2,xdims,nargout=2)  #grid at t=0
    eng.quit()                                                #close the matlab instance

    X_r = np.array(X_r)                                       #Points and weights are brought into the correct type and
    w_r = np.array(w_r)                                       #shape to work here
    w_r = w_r.flatten()
    X_r = tf.constant(np.transpose(X_r),dtype=DTYPE)

    X_b = np.array(X_b)
    w_b = np.array(w_b)
    w_b = w_b.flatten()
    X_b = tf.constant(np.transpose(X_b),dtype=DTYPE)

    X_0 = np.array(X_0)
    w_0 = np.array(w_0)
    w_0 = w_0.flatten()
    X_0 = tf.constant(np.transpose(X_0),dtype=DTYPE)

    return X_r, w_r, X_b, w_b, X_0, w_0

## Testing hyperparameters

For testing hyperparameters, a simple product grid quadrature shall be used. This is set up here.

In [ ]:
level = 10            #size of product grid to be used

N_r = level**(xdims+1)       #resulting number of internal points
N_0 = level**xdims           #number of points for initial condition
N_b = 2*xdims*level**xdims   #number of points for boundary condition

#quadrature on [-1,1]^d:
X_0 = tf.ones((N_0,1), dtype=DTYPE)*lbt         #t=0 for initial condition
w_0 = np.ones(N_0)
for i in range(xdims):                          #full grid over spatial dimensions
    pts,wts = leja_tensor_pts(i,xdims,level)    #the grid is built column by column using the function written before
    X_0 = tf.concat([X_0, (lbx+(ubx - lbx)/2) + (ubx - lbx)/2 * pts],1) #scaling to the spatial interval
    w_0 = w_0 * wts
    
#quadrature on the boundary:
X_b, w_b = leja_tensor_boundary_pts(0,level)      #using the previously written function to get points and weights
for i in range(1,xdims):                          #on all faces of the boundary
    pts, wts = leja_tensor_boundary_pts(i,level)
    X_b = tf.concat([X_b, pts],0)
    w_b = w_b * wts

#quadrature on the entire domain
pts, w_r = leja_tensor_pts(0,xdims+1,level)
X_r = (lbt+(ubt - lbt)/2) + (ubt - lbt)/2 * pts   #covering [0,1] for the time
for i in range(xdims):                            #the grid is built column by column using the function written before
    pts, wts = leja_tensor_pts(i+1,xdims+1,level)
    X_r = tf.concat([X_r, (lbx+(ubx - lbx)/2) + (ubx - lbx)/2 * pts],1) #scaling to the spatial interval
    w_r = w_r * np.array(wts)

    
w_0 = tf.constant(np.array(w_0),dtype=DTYPE)
w_b = tf.constant(np.array(w_b),dtype=DTYPE)
w_r = tf.constant(np.array(w_r),dtype=DTYPE)

u_0 = fun_u_0(X_0)     #values for initial
u_b = fun_u_b(X_b)     #and boundary condition

X_data = [X_0, X_b]    #summarizing (for initial and boundary condition) points used
u_data = [u_0, u_b]    #expected values there
w_data = [w_0, w_b]    #and quadrature weights

### Network architecture

We choose which sizes of networks should be tested.

In [ ]:
widths = [5,10,20]   #which widths and depths of the network to test
depths = [2,4,8]     #all combinations will be explored

net_reps = 3             #since the initialization of weights and biases is random, multiple passes can be made to
                         #mitigate the randomness

results = np.empty((net_reps,len(widths),len(depths)))   #results will be written in this
params = np.empty((2,len(widths),len(depths)))           #alongside wigth and depth in this

Now we run the optimization for all those configurations.

In [ ]:
from time import time  #for time measurements

t0_total = time() #total time will be measured

config_num = 1 
models = []       #all the PINNs will be saved here

w=0
for width in widths:
    d=0
    for depth in depths:
        print("exploring network config {} of {}\n".format(config_num,len(widths)*len(depths)))
        config_num = config_num +1
        
        t0 = time() #time spent on this configuration will be measured
        j = 0
        
        for net_rep in range(net_reps):

                #creating a new network with the desired depth and width:
                models.append(init_model(num_hidden_layers=depth, num_neurons_per_layer=width))
                
                tstart = time()  #to measure time of this one pass

                hist = []

                for i in range(N_adam):   #Adam part of optimizing
                    loss = train_step(models[-1],X_r, w_r, X_data, u_data, w_data)
                    hist.append(loss.numpy())
                    if loss.numpy()<e_target_adam:
                        break
        
                    if i%100 == 0:
                        print('It {:05d}: loss = {:6.4e}'.format(i,loss))
    
                #BFGS part of optimizing:
                train_with_scipy_optimizer(models[-1], X_r, w_r, X_data, u_data, w_data, hist)                

                j = j+1
                
                #output for control:
                print('\n{} of {} done, computation time: {} seconds'.format(j,net_reps,time()-tstart))
                print('Total in this config so far: {} seconds\n'.format(time()-t0))

                err_two = get_error(models[-1])  #final error of this PINN

                results[net_rep,w,d] = err_two
                params[0,w,d] = width
                params[1,w,d] = depth
        

        print("\ntotal time so far: {}\n".format(time()-t0_total))

        
        d = d+1
    w = w+1

If multiple passes have been used we can now look at their mean as well as their minimum.

In [ ]:
num_widths = np.shape(results)[1]
num_depths = np.shape(results)[2]


def print_with_params(result,params,precision):                  #this will print widths and depths used as the first
    result_with_params = np.empty((num_widths+1,num_depths+1))   #line/column

    result_with_params[1:num_widths+1,1:num_depths+1] = result
    result_with_params[0,1:num_depths+1] = params[1,0,:]
    result_with_params[1:num_widths+1,0] = params[0,:,0]
    result_with_params[0,0] = 0

    T = result_with_params.T

    print(T.round(precision))

def print_means(results,params,precision):     #mean of all passes
    means = np.mean(results,axis=0)

    print_with_params(means,params,precision)


def print_mins(results,params,precision):      #minimum of all passes
    mins = np.min(results,axis=0)

    print_with_params(mins,params,precision)


def print_variance(results,params,precision):  #observed variance
    means = np.mean(results,axis=0)

    variance = np.zeros(means.shape)

    for i in range(net_reps):
        new_var = np.square(means-results[i,:,:])
        variance = variance + new_var

    print_with_params(variance/(net_reps-1),params,precision)
    

np.set_printoptions(suppress=True)

print("mean: \n")
print_means(results,params,5)

print("\nminimum: \n")
print_mins(results,params,5)

### Optimization

This section will explore the convergence of the combination of Adam and L-BFGS-G. For this, the optimizer is simply run once.

In [ ]:
model = init_model()

hist = []      #saving loss history
err_hist = []  #saving error history

for i in range(N_adam):                                             #Adam training steps
    loss = train_step(model, X_r, w_r, X_data, u_data, w_data)
    hist.append(loss.numpy())
    err_hist.append(get_error(model))
    if loss.numpy()<e_target_adam:
        break
        
    if i%100 == 0:
        print('It {:05d}: loss = {:6.4e}'.format(i,loss))

adam_num_steps = len(hist)
    
train_with_scipy_optimizer(model, X_r, w_r, X_data, u_data, w_data, hist, err_hist=err_hist)  #BFGS training

print("\ndone")

We can plot the convergence now.

In [ ]:
import matplotlib.pyplot as plt

iters = range(len(hist));

fig = plt.figure(figsize=(8,4))
ax = fig.add_subplot(111)

ax.semilogy(iters[:adam_num_steps+1],hist[:adam_num_steps+1],'b-', label="Loss")        #loss while using Adam
ax.semilogy(iters[adam_num_steps+2:],hist[adam_num_steps+2:],'c-', label="Loss")        #loss while using BFGS

ax.semilogy(iters[:adam_num_steps+1],err_hist[:adam_num_steps+1],'r-', label="Error")  #error while using Adam
ax.semilogy(iters[adam_num_steps+2:],err_hist[adam_num_steps+2:],'m-', label="Error")  #error while using BFGS

ax.set_xlabel('Iteration')
plt.legend()
plt.show()

### Weights in the loss function

As defined before, the loss function can take weights, it is of the form $$\lambda_{int} err_{int} + \lambda_{sb} err_{sb}
+ \lambda_{ab} err_{tb}.$$ Several tests can be conducted here. The first kind is weighting where $\lambda_{int} + \lambda_{sb} + \lambda_{ab} = 3$ remains fixed and we vary one of the weights (e.g. $\lambda_{int}$) while scaling the others accordingly (in that example $\lambda_{sb} = \lambda_{tb} = \frac{3-\lambda_{int}}{2}$). The second kind is general scaling of the loss function by $\lambda_{int} = \lambda_{sb} = \lambda_{ab} = \lambda$.

In [ ]:
lams = np.arange(0.1,2.9,0.2)      #for testing weighting
#lams = [1,3,10,30,100,300,1000,3000,10000]  #for testing scaling

In accordance with what what is supposed to be tested, the correct lines need to be activated in the following cell. Then, the cell can be executed to perform the optimization for all the chosen weightings/scalings.

In [ ]:
from time import time  #for time measurements

lam_errs = []

model = init_model()
start_weights = model.get_weights()    #save starting weights


cur_lam = 1

for lam in lams:
    model.set_weights(start_weights)   #reapply starting weights for fairness between runs

    
    tstart = time() #for measuring time of this run

    hist = []
    err_hist = []

    for i in range(N_adam):
        loss = train_step(model, X_r, w_r, X_data, u_data, w_data,
                          lam_int = lam, lam_sb = 0.5*(3-lam), lam_tb = 0.5*(3-lam))  #for testing weighting of lam_int
                          #lam_sb = lam, lam_int = 0.5*(3-lam), lam_tb = 0.5*(3-lam))  #for testing weighting of lam_sb
                          #lam_int = lam, lam_sb = lam, lam_tb = lam)                 #for testing scaling
    
        hist.append(loss.numpy())
        err_hist.append(get_error(model))
        if loss.numpy()<(e_target_adam):      #for testing weighting
        #if loss.numpy()<(e_target_adam*lam): #for testing scaling
            break
        
        if i%100 == 0:
            print('It {:05d}: loss = {:6.4e}'.format(i,loss))
    
    train_with_scipy_optimizer(model, X_r, w_r, X_data, u_data, w_data, hist, err_hist = err_hist,
                          lam_int = lam, lam_sb = 0.5*(3-lam), lam_tb = 0.5*(3-lam))  #for testing weighting of lam_int
                          #lam_sb = lam, lam_int = 0.5*(3-lam), lam_tb = 0.5*(3-lam))  #for testing weighting of lam_sb
                          #lam_int = lam, lam_sb = lam, lam_tb = lam)                 #for testing scaling
    lam_errs.append(min(err_hist))

    print('lam {} of {} done, time {}'.format(cur_lam,len(lams),time()-tstart))
    cur_lam = cur_lam+1

We can now plot the results. Once again, the correct lines should be selected depending on the testing done.

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(8,5))
ax = fig.add_subplot(111)

ax.semilogy(lams,lam_errs,'bo')     #when weighting has been tested
#ax.loglog(lams,lam_errs,'bo')      #when scaling has been tested

ax.set_xlabel('$\lambda_{int}$')    #when weighting of lam_int has been tested
#ax.set_xlabel('$\lambda_{sb}$')    #when weighting of lam_sb has been tested
#ax.set_xlabel('$\lambda$')         #when scaling has been tested
ax.set_ylabel('Error (PINN)')       
plt.show()

## Quadrature formulas

First, we will set which quadrature formulas to use and set up the neural network. We also prepare list to save the number of points in each grid.

In [ ]:
start_level = 6     #levels for tensor product (=points per dimension)
level_step = 1
step_num = 6
levels = range(start_level,start_level + step_num*level_step, level_step)

sparse_start = 8    #levels for sparse grid
sparse_step = 1
sparse_step_num = 7
sparse_levels = range(sparse_start,sparse_start + sparse_step_num*sparse_step, sparse_step)

reps = 3   #number of repetitions of monte carlo

#network:
model = init_model()
start_weights = model.get_weights()

N_r_list = []
N_0_list = []  #number of points used will go here
N_b_list = []

Then, we can set up  all the quadrature formulas and perform the optimization for all of them. We start with the product quadrature rules.

In [ ]:
from time import time  #for time measurements

loss_leja = []
err_leja_two = []

j = 0;
for level in levels:             #using the selected sizes of product grid

    N_r = level**(xdims+1)
    N_0 = level**xdims           #we compute the number of points used
    N_b = 2*xdims*level**xdims
    
    N_r_list.append(N_r)
    N_0_list.append(N_0)         #and save those numbers for later use in Monte Carlo
    N_b_list.append(N_b)

    #quadrature on [-1,1]^d:
    X_0 = tf.ones((N_0,1), dtype=DTYPE)*lbt         #t=0
    w_0 = np.ones(N_0)
    for i in range(xdims):                          #product grid over the spatial dimensions
        pts,wts = leja_tensor_pts(i,xdims,level)
        X_0 = tf.concat([X_0, (lbx+(ubx - lbx)/2) + (ubx - lbx)/2 * pts],1)
        w_0 = w_0 * wts
    
    #quadrature on the boundary:
    X_b, w_b = leja_tensor_boundary_pts(0,level)
    for i in range(1,xdims):                           #collecting the points of all faces of the boundary
        pts, wts = leja_tensor_boundary_pts(i,level)
        X_b = tf.concat([X_b, pts],0)
        w_b = w_b * wts
    
    #quadrature over the entire domain:
    pts, w_r = leja_tensor_pts(0,xdims+1,level)
    X_r = (lbt+(ubt - lbt)/2) + (ubt - lbt)/2 * pts #t
    for i in range(xdims):                             #full product grid
        pts, wts = leja_tensor_pts(i+1,xdims+1,level)
        X_r = tf.concat([X_r, (lbx+(ubx - lbx)/2) + (ubx - lbx)/2 * pts],1)
        w_r = w_r * np.array(wts)

    w_0 = tf.constant(np.array(w_0),dtype=DTYPE)
    w_b = tf.constant(np.array(w_b),dtype=DTYPE)
    w_r = tf.constant(np.array(w_r),dtype=DTYPE)

    u_0 = fun_u_0(X_0)   #initial data
    u_b = fun_u_b(X_b)   #boundary data

    X_data = [X_0, X_b]  #collecting data into one variable
    u_data = [u_0, u_b]
    w_data = [w_0, w_b]

    #reset the weights for fairness between runs:
    model.set_weights(start_weights)

    tstart = time()  #for measuring time of this run

    hist = []

    for i in range(N_adam):         #training with Adam
        loss = train_step(model,X_r, w_r, X_data, u_data, w_data)
        hist.append(loss.numpy())
        if loss.numpy()<e_target_adam:
            break
        
        if i%1000 == 0:
            print('It {:05d}: loss = {:6.4e}'.format(i,loss))
    
    train_with_scipy_optimizer(model, X_r, w_r, X_data, u_data, w_data, hist)    #training with BFGS            

    j = j+1
    #output time for this run and final loss:
    print('\n{} of {} product grids done, computation time: {} seconds'.format(j,step_num,time()-tstart))
    print('Final loss: {}'.format(hist[-1]))

    err_two = get_error(model)      #compute L2-error of the PINN

    loss_leja.append(hist[-1])      #save loss and error of this run
    err_leja_two.append(err_two)

Second are the sparse grids.

In [ ]:
loss_sparse = []
err_sparse_two = []

j = 0
for level in sparse_levels:    
    X_r, w_r, X_b, w_b, X_0, w_0 = leja_sparse_pts(level) #sparse grids are imported from Matlab

    N_r_list.append(X_r.shape[0])
    N_b_list.append(X_b.shape[0])  #once again, the numbers of grid points are saved for later use
    N_0_list.append(X_0.shape[0])

    u_0 = fun_u_0(X_0) #initial data
    u_b = fun_u_b(X_b) #boundary data

    X_data = [X_0, X_b]  #collecting data into one variable
    u_data = [u_0, u_b]
    w_data = [w_0, w_b]

    #reset the weights for fairness between runs:
    model.set_weights(start_weights)

    tstart = time()  #for measuring time of this run

    hist = []

    for i in range(N_adam):         #training with Adam
        loss = train_step(model,X_r, w_r, X_data, u_data, w_data)
        hist.append(loss.numpy())
        if loss.numpy()<e_target_adam:
            break
        
        if i%1000 == 0:
            print('It {:05d}: loss = {:6.4e}'.format(i,loss))
    
    train_with_scipy_optimizer(model, X_r, w_r, X_data, u_data, w_data, hist)    #training with BFGS             

    j = j+1
    #output time for this run and final loss:
    print('\n{} of {} sparse grids done, computation time: {} seconds'.format(j,sparse_step_num,time()-tstart))
    print('Final loss: {}'.format(hist[-1]))

    err_two = get_error(model) #compute L2-error of the PINN

    loss_sparse.append(hist[-1])  #save loss and error of this run
    err_sparse_two.append(err_two)

Third, Monte Carlo will be used.

In [ ]:
all_rand_errs = np.ones((reps,step_num+sparse_step_num))
loss_random = []
err_random_two = []

j = 0
li = 0
for run in range(len(N_r_list)):

    N_r = N_r_list[run]
    N_b = N_b_list[run]    #using grid sizes from before
    N_0 = N_0_list[run]

    rand_losses = []
    rand_errs_two = []
        
    for r in range(reps):
        #quadrature on [-1,1]^d:
        X_0 = tf.ones((N_0,1), dtype=DTYPE)*lbt         #t=0
        X_0 = tf.concat([X_0, tf.random.uniform((N_0,xdims), lbx, ubx, dtype=DTYPE)],1) #random points otherwise
        w_0 = tf.constant(np.ones(N_0)/N_0,dtype=DTYPE) #all weights are the same for monte carlo
        
        #quadrature on the boundary:
        X_b = random_boundary_pts(0, N_b//xdims)
        for i in range(1,xdims):                  #points from each face of the boundary as defined before
            X_b = tf.concat([X_b, random_boundary_pts(i, N_b//xdims)],0)
        w_b = tf.constant(np.ones(N_b)/N_b,dtype=DTYPE)
        
        #quadrature on the entire domain:
        X_r = tf.random.uniform((N_r,1), lbt, ubt, dtype=DTYPE) #t
        X_r = tf.concat([X_r, tf.random.uniform((N_r,xdims), lbx, ubx, dtype=DTYPE)], 1)    #x
        w_r = tf.constant(np.ones(N_r)/N_r,dtype=DTYPE)


        u_0 = fun_u_0(X_0)   #initial data
        u_b = fun_u_b(X_b)   #boundary data

        X_data = [X_0, X_b]  #collecting data into one variable
        u_data = [u_0, u_b]
        w_data = [w_0, w_b]

        #reset the weights for fairness between runs:
        model.set_weights(start_weights)

        tstart = time()  #for measuring time of this run

        hist = []
        for i in range(N_adam):      #training with Adam
            loss = train_step(model,X_r, w_r, X_data, u_data, w_data)
            if loss.numpy()<e_target_adam:
                break
            if i%1000 == 0:
                print('It {:05d}: loss = {:6.4e}'.format(i,loss))
        
        train_with_scipy_optimizer(model, X_r, w_r, X_data, u_data, w_data, hist) #training with BFGS

        j = j+1
        #output time for this run and final loss:
        print('\n{} of {} random grids done, computation time: {} seconds'.format(j,len(N_r_list)*reps,time()-tstart))
        print('Final loss: {}'.format(hist[-1]))
        
        err_two = get_error(model) #compute L2-error of the PINN 

        rand_losses.append(hist[-1])  #save loss and error of this run
        rand_errs_two.append(err_two)
        
        all_rand_errs[r,li] = err_two #errors of every run are saved here
        
    loss_random.append(np.mean(rand_losses))       #mean of all losses with this number of points
    err_random_two.append(np.mean(rand_errs_two))  #mean of all errors with this number of points

    li = li+1

Finally, we plot the results with respect to the number of points used in the quadrature rule ofer the entire domain.

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(8,5))
ax = fig.add_subplot(111)

for r in range(reps):
    ax.semilogy(N_r_list,all_rand_errs[r,:],'ko', alpha=0.25)

ax.semilogy(N_r_list,err_random_two,'ko', label="Monte Carlo")
ax.semilogy(N_r_list[0:step_num],err_leja_two,'bo', label="Leja (product)")
ax.semilogy(N_r_list[step_num:step_num+sparse_step_num],err_sparse_two,'ro', label='Leja (sparse)')

ax.set_xlabel('$N_r$')
ax.set_ylabel('Final $L_2$-error of the PINN');
plt.legend()
plt.show()